In [8]:
import json
import requests
from tqdm import tqdm
import pandas as pd
import numpy as np
from collections import Counter
from concurrent.futures import ThreadPoolExecutor

import nltk
import spacy

# Set your keywords
my_input = 'salinization AND flooding'
# Set your API Key
key = 'a4dfe1b4f9535e6e41587a40f9ba9877'

# Create a session for making requests
session = requests.Session()
session.headers['X-ELS-APIKey'] = key
session.headers['X-ELS-ResourceVersion'] = 'XOCS'
session.headers['Accept'] = 'application/json'

def scopus_search(my_input: str) -> list:
    api_resource = "https://api.elsevier.com/content/search/scopus?"
    search_param = f'query=title-abs-key({my_input})'  # for example

    # Set the desired number of results per page
    results_per_page = 25

    # Send the first request to get the total number of results
    first_page_request = session.get(api_resource + search_param + f"&count={results_per_page}&start=0")
    first_page = json.loads(first_page_request.content.decode("utf-8"))

    total_results = int(first_page['search-results']['opensearch:totalResults'])
    total_pages = (total_results // results_per_page) + 1

    # List to store all articles
    articles_list = []

    print(f"Scrapping Data Pages from Scopus using {my_input}...")
    # Iterate over all pages
    with ThreadPoolExecutor() as executor:
        for page_number in tqdm(range(total_pages)):
            start_index = page_number * results_per_page
            page_request = session.get(api_resource + search_param + f"&count={results_per_page}&start={start_index}")
            page = json.loads(page_request.content.decode("utf-8"))
            try:
                articles_list.extend(page['search-results']['entry'])
            except:
                continue

        print(f"Number of articles: {len(articles_list)}")
        return articles_list

In [ ]:

article_title = []
article_doi = []
article_eid = []
article_ID = []
article_pii = []
article_url = []
article_lead= []
article_pub = []
article_coverDate = []

# Access individual articles
with ThreadPoolExecutor() as executor:
    for article in tqdm(range(len(articles_list))):
        try:
            article_pii.append(articles_list[article].get("pii"))
            article_title.append(articles_list[article].get("dc:title"))
            article_doi.append(articles_list[article].get("prism:doi"))
            article_eid.append(articles_list[article].get("eid"))
            article_ID.append(articles_list[article].get("dc:identifier"))
            article_url.append(articles_list[article].get("prism:url"))
            article_lead.append(articles_list[article].get("dc:creator"))
            article_pub.append(articles_list[article].get("prism:publicationName"))
            article_coverDate.append(articles_list[article].get("prism:coverDate"))

        except:
            article_pii.append(None)
            article_doi.append(None)
            article_title.append(articles_list[article].get("dc:title"))
            article_eid.append(articles_list[article].get("eid"))
            article_ID.append(articles_list[article].get("dc:identifier"))
            article_url.append(articles_list[article].get("prism:url"))
            article_lead.append(None)
            article_pub.append(articles_list[article].get("prism:publicationName"))
            article_coverDate.append(articles_list[article].get("prism:coverDate"))


In [51]:
def scopus_id_abstract_retriever(scopus_id: str) -> str:
    api_endpoint = f"https://api.elsevier.com/content/abstract/scopus_id/{scopus_id}"

    # Make the request to retrieve the abstract
    response = session.get(api_endpoint)
    data = json.loads(response.content.decode("utf-8"))
    # Extract the abstract from the response
    try:
        abstract = data["abstracts-retrieval-response"]["coredata"]["dc:description"]
        affiliation = data["abstracts-retrieval-response"]["affiliation"]['affilname']
        # Study Area
        area = data["abstracts-retrieval-response"]["subject-areas"]["subject-area"]
        subjects = [subject["$"] for subject in area]
        result = " & ".join(subjects)
        # Authors
        author_count = len(data["abstracts-retrieval-response"]["authors"]['author'])

    except:
        abstract = "NA"
        affiliation = data["abstracts-retrieval-response"]["affiliation"]['affilname']
        # Study Area
        area = data["abstracts-retrieval-response"]["subject-areas"]["subject-area"]
        subjects = [subject["$"] for subject in area]
        result = " & ".join(subjects)
        # Authors
        author_count = len(data["abstracts-retrieval-response"]["authors"]['author'])


    # Return the abstract
    return abstract, affiliation, result, author_count

In [52]:
scopus_id_abstract_retriever("SCOPUS_ID:85151557628")

('Saline-alkali land, a precious candidate arable land resources, plays a critical role in achieving agricultural sustainability. Drip irrigation (DI) is an effective method for rationalizing of saline-alkali land. Nevertheless, the inapposite application of DI increases the risk of secondary salinization, significantly leading to severe soil degradation and yield decline. In this study, we conducted a meta-analysis to quantify the impacts of DI on soil salinity and crop yield to determine the appropriate DI management strategies for an irrigated agricultural system in saline-alkali land. The results showed that DI generally decreased soil salinity in the root zone by 37.7 % and increased crop yield by 37.4 % relative to flooding irrigation (FI). Drip emitters with a flow rate of 2–4 L h−1 were recommended to obtain positive effects on soil salinity control and agricultural production when an irrigation quota was below 50 % crop evapotranspiration (ETc), and the salinity of irrigation 